# Boston Housing Classification SVM Cross Validation

In [ ]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import expr
from pyspark.sql.session import SparkSession
from pyspark.sql.types import BooleanType
from helpers.helper_functions import translate_to_file_string

In [ ]:
inputFile = translate_to_file_string("../data/Boston_Housing_Data.csv")

Spark session creation 

In [ ]:
spark = (SparkSession
       .builder
       .appName("BostonHousingClassSVMCV")
       .getOrCreate())

DataFrame creation using an ifered Schema 

In [ ]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile) \
       .withColumn("CATBOOL", expr("CAT").cast(BooleanType()))
print(df.printSchema())

In [ ]:
splits = df.randomSplit([0.9, 0.1 ], 12345)
training = splits[0]
test = splits[1]

Prepare training and test data.

Data preprocessing

In [ ]:
featureCols = df.columns.copy()
featureCols.remove("MEDV")
featureCols.remove("CAT")
featureCols.remove("CATBOOL") 
print(featureCols)

assembler =  VectorAssembler(outputCol="features", inputCols=featureCols)

Build the evaluator

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="CAT",rawPredictionCol="rawPrediction", metricName="areaUnderROC")

Support Vector Machine Classifier

In [ ]:
lsvc = LinearSVC(labelCol="CAT",aggregationDepth=2, featuresCol="features" ) 

Build the pipeline

In [ ]:
pipeline = Pipeline(stages= [assembler, lsvc] )

Build the paramGrid

In [ ]:
# TODO add additional param values for a nested cross validation
# TODO add additional params 
paramGrid = ParamGridBuilder().addGrid(lsvc.maxIter, [100])\
                                 .addGrid(lsvc.regParam, [0.1]) \
                                 .build()

Build the CrossValidator 

In [ ]:
#TODO change the params of the cross validation to improve the result
cvSVM = CrossValidator(estimator=pipeline, evaluator=evaluator, \
                          estimatorParamMaps=paramGrid, numFolds=2, parallelism=2)

Train the model 

In [ ]:
cvSVMModel = cvSVM.fit(training)

Show best Model 

In [ ]:
linearSVCModel = cvSVMModel.bestModel.stages[1] # the stage at index 1 in the pipeline is the SVMModel
print("Best Params: \n", linearSVCModel.explainParams())
print("Param Map: \n", linearSVCModel.extractParamMap())

Test the model

In [ ]:
predictions = cvSVMModel.transform(test)
predictions.show()

In [ ]:
accuracy = evaluator.evaluate(predictions)
print("Test Error",(1.0 - accuracy))